In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import datetime 
import re

In [3]:
url_MSB = 'https://www.msb.com.vn/vi/w/ca-nhan/tiet-kiem/tiet-kiem-co-ky-han-truc-tuyen' #done
url_OCB = 'https://ocb.com.vn/vi/cong-cu/lai-suat' #done
url_VCB = 'https://www.vietcombank.com.vn/vi-VN/KHCN/Cong-cu-Tien-ich/KHCN---Lai-suat' #done

url_BID = 'https://bidv.com.vn/vn/tra-cuu-lai-suat' # Nun
url_MBB = 'https://www.mbbank.com.vn/Fee' # Nun
url_TPB = 'https://tpb.vn/cong-cu-tinh-toan/lai-suat' 
url_VIB = 'https://www.vib.com.vn/vn/tiet-kiem/bieu-lai-suat-tiet-kiem-tai-quay'

In [4]:
# Define the column names as a list
columns = ['MMYYYY','Bank_name', 'CASA_int', 'Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y', 'Onl_1M', 'Onl_3M', 'Onl_6M','Onl_1Y','Onl_2Y']
Update_inst = pd.DataFrame(columns=columns)

MSB_page = requests.get (url_MSB)

MSB_soup = BeautifulSoup(MSB_page.text, 'html') 
MSB_table = MSB_soup.find('table')
body = MSB_table.find('tbody')
column_data = body.find_all ('tr')

offline_values = []
online_values = []
for x in [0,1,3,6,12,16]:
    offline = body.find_all ('tr')[x].find_all('td')[2].text.strip() 
    online = body.find_all ('tr')[x].find_all('td')[3].text.strip() 
    offline_values.append(offline)
    online_values.append(online)

new_row = {
    'MMYYYY': datetime.datetime.now().strftime("%d/%m/%Y"),
    'Bank_name': 'MSB',
    'CASA_int': offline_values[0],
    'Off_1M': offline_values[1],
    'Off_3M': offline_values[2],
    'Off_6M': offline_values[3],
    'Off_1Y': offline_values[4],
    'Off_2Y': offline_values[5],
    'Onl_1M': online_values[1],
    'Onl_3M': online_values[2],
    'Onl_6M': online_values[3],
    'Onl_1Y': online_values[4],
    'Onl_2Y': online_values[5],
}
for column in ['CASA_int', 'Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y', 'Onl_1M', 'Onl_3M', 'Onl_6M','Onl_1Y','Onl_2Y']:
    new_row[column] = float(new_row[column].replace('%', ''))
    new_row_ctg = pd.DataFrame([new_row])
Update_inst = pd.concat([Update_inst, new_row_ctg], ignore_index=True)
Update_inst

AttributeError: 'NoneType' object has no attribute 'find'

In [5]:
# Gửi yêu cầu GET để lấy nội dung HTML
response = requests.get(url_OCB)

# Kiểm tra nếu yêu cầu thành công
if response.status_code == 200:
    # Sử dụng BeautifulSoup để phân tích cú pháp HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Tìm bảng có class là "text-right text-nowrap" (hoặc class của bạn)
    table = soup.find('tbody')

    # Tạo danh sách trống để lưu dữ liệu
    data = []

    # Lặp qua các hàng trong bảng
    for row in table.find_all('tr'):
        # Lấy tất cả các cột trong mỗi hàng
        columns = row.find_all('td')
        # Lấy nội dung của các cột và lưu vào danh sách dữ liệu
        data.append([column.get_text() for column in columns])

    # Chuyển danh sách dữ liệu thành DataFrame của pandas
    df = pd.DataFrame(data, columns=['Kỳ hạn', 'Other', 'Offline', 'Online'])

    # In ra bảng dữ liệu
    print(df)

else:
    print('Yêu cầu không thành công.')

[['Không kì hạn', '0.1', '0.1', '0.1'],
 ['1 Tháng', '3.9', '3.9', '4.1'],
 ['3 Tháng', '4.1', '4.1', '4.25'],
 ['6 Tháng', '5.1', '5.1', '5.3'],
 ['9 Tháng', '5.2', '5.2', '5.4'],
 ['12 Tháng', '5.5', '5.5', '5.5'],
 ['13 Tháng', '5.5', '5.5', '5.5'],
 ['15 Tháng', '5.5', '5.5', '5.5'],
 ['18 Tháng', '5.9', '5.9', '5.9'],
 ['21 Tháng', '5.9', '5.9', '5.9'],
 ['24 Tháng', '6', '6', '6'],
 ['36 Tháng', '6.1', '6.1', '6.1']]

In [36]:
new_row = {
    'MMYYYY': datetime.datetime.now().strftime("%d/%m/%Y"),
    'Bank_name': 'OCB',
    'CASA_int': df.loc[df['Kỳ hạn'] == 'Không kì hạn', 'Offline'].values[0],
    'Off_1M': df.loc[df['Kỳ hạn'] == '1 Tháng', 'Offline'].values[0],
    'Off_3M': df.loc[df['Kỳ hạn'] == '3 Tháng', 'Offline'].values[0],
    'Off_6M': df.loc[df['Kỳ hạn'] == '6 Tháng', 'Offline'].values[0],
    'Off_1Y': df.loc[df['Kỳ hạn'] == '9 Tháng', 'Offline'].values[0],
    'Off_2Y': df.loc[df['Kỳ hạn'] == '24 Tháng', 'Offline'].values[0],
    'Onl_1M': df.loc[df['Kỳ hạn'] == '1 Tháng', 'Online'].values[0],
    'Onl_3M': df.loc[df['Kỳ hạn'] == '3 Tháng', 'Online'].values[0],
    'Onl_6M': df.loc[df['Kỳ hạn'] == '6 Tháng', 'Online'].values[0],
    'Onl_1Y': df.loc[df['Kỳ hạn'] == '9 Tháng', 'Online'].values[0],
    'Onl_2Y': df.loc[df['Kỳ hạn'] == '24 Tháng', 'Online'].values[0],
}

# Chuyển đổi các giá trị thành số thập phân (loại bỏ %)
for column in ['CASA_int', 'Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y', 'Onl_1M', 'Onl_3M', 'Onl_6M', 'Onl_1Y', 'Onl_2Y']:
    new_row[column] = float(new_row[column].replace('%', ''))

# Tạo DataFrame từ dữ liệu mới
new_row_ocb = pd.DataFrame([new_row])

# Nếu Update_inst là DataFrame ban đầu, thêm dòng mới vào nó
Update_inst = pd.concat([Update_inst, new_row_ocb], ignore_index=True)

In [37]:
Update_inst

,MMYYYY,Bank_name,CASA_int,Off_1M,Off_3M,Off_6M,Off_1Y,Off_2Y,Onl_1M,Onl_3M,Onl_6M,Onl_1Y,Onl_2Y
0,17/10/2023,OCB,0.1,3.9,4.1,5.1,5.2,6.0,4.1,4.25,5.3,5.4,6.0


In [38]:
x = requests.get("https://www.vietcombank.com.vn/vi-VN/KHCN/Cong-cu-Tien-ich/KHCN---Lai-suat")
html = x.text

start = html.find("currentDataInterestRate")
end = html.find("}]}")
html_2= html[start:end]

matches = re.findall(r'&quot;rates&quot;:([\d.]+)', html_2)
df = [m[:5] for m in matches]


Update_inst.loc[1] = Update_inst.loc[0]  # Copy data from row 0 to row 1
Update_inst.loc[0, 'MMYYYY'] = datetime.datetime.now().strftime("%d/%m/%Y")
Update_inst.loc[0, 'Bank_name'] = 'VCB'
Update_inst.loc[0, 'CASA_int'] = df[0]
Update_inst.loc[0, 'Off_1M'] = df[9]
Update_inst.loc[0, 'Off_3M'] = df[15]
Update_inst.loc[0, 'Off_6M'] = df[18]
Update_inst.loc[0, 'Off_1Y'] = df[24]
Update_inst.loc[0, 'Off_2Y'] = df[27]
Update_inst.loc[0, 'Onl_1M'] = df[70]
Update_inst.loc[0, 'Onl_3M'] = df[71]
Update_inst.loc[0, 'Onl_6M'] = df[72]
Update_inst.loc[0, 'Onl_1Y'] = df[76]
Update_inst.loc[0, 'Onl_2Y'] = df[79]



In [39]:
Update_inst

,MMYYYY,Bank_name,CASA_int,Off_1M,Off_3M,Off_6M,Off_1Y,Off_2Y,Onl_1M,Onl_3M,Onl_6M,Onl_1Y,Onl_2Y
0,17/10/2023,VCB,0.001,0.03,0.033,0.043,0.053,0.053,0.03,0.033,0.043,0.053,0.053
1,17/10/2023,OCB,0.1,3.9,4.1,5.1,5.2,6.0,4.1,4.25,5.3,5.4,6.0
